# Input vector construction and Harvest prediction for all Heddingle fields from data in a single year

In [39]:
# In particular we now include the correct dates for weather grouping for hostvete 
# which begin from seed date -> 1st Nov year before + 15Marh->midsummer + midsummer -> max harvest date

# We load all soil, field, year and weather data for all fields in Heddinge and predict 
# the harvest for that field during a single year: 2017, 2018, 2019 or 2020

# Also change the grouping in time for weather data to just a single season (Aug 2)
# instead of the previous weekly grouping

# We segment per crop and year in this particular notebook (July 31, Aug 1)

# First start the VPN in order to download all necessary data 

In [53]:
# Function to load a list with all properties from a particular field
def loadfieldproperties(datain):
    fieldattr=[]
    fieldkeys=[]
    for key in datain["properties"].keys():
        #print(singlefield["properties"][key])
        fieldattr.append(datain["properties"][key])
        fieldkeys.append(key)
    return fieldattr, fieldkeys

In [54]:
def keystolist(dictin):
    x=[]
    y=[]
    for key in dictin.keys():
        x.append(key)
        y.append(dictin[key])
    return x, y

In [56]:
# loading a pickled (saved) list 
def loadfield(path_to_file, fieldID):
  file_name = path_to_file+fieldID#+'.pkl'
  open_file = open(file_name, "rb")
  loaded_list = pickle.load(open_file)
  open_file.close()

  return loaded_list

In [57]:
# Function to load a list with all properties from a particular field
def loadfieldpropertiestoDF(datain):

    fieldattr = []
    fieldkeys = []
    for key in datain["properties"].keys():
        fieldattr.append(datain["properties"][key])
        fieldkeys.append(key)

    return pd.DataFrame([fieldattr], columns = fieldkeys) #tempDF #fieldattr, fieldkeys

In [58]:
# Loads all subfield data from a given field ID into lists
def loadintodf(maindf, result):

    subfieldIDs = len((result["SRData"][0])["field"])
    #print("There are "+str(subfieldIDs)+" different fields within that field ID.")
    #print("These are: ")
    #templist = []
    for k in range(subfieldIDs):
        print(k)
        #print("ID: ",(result["SRData"][0])["field"][k]["id"])
        #flist, fkeys = 
        tempdf = LoadFieldDataToDF(result, k) #loadfielddatatodf(result,k)
        
        #templist.append(flist)
        maindf = maindf.append(tempdf)
    #templist.insert(0, fkeys)
    
    return maindf # templist

In [60]:
# Creates a dataframe from a keylist and a datalist
def ListstoDF(keylist, datalist):
  return pd.DataFrame([datalist], columns=keylist)

In [61]:
def ListfromData(indata, flag):

    datalist = []
    if flag == 0:
        keyslist = ['id', 'coordinates']
        datalist.append(indata['id']) # id data
        coord = indata['geometry']['coordinates']
        conv_data = coord # ConvUTM33NToWGS84(coord)
        datalist.append(conv_data)       # geometry
    elif flag  == 2: # For harvest
        keyslist = ['id', 'coordinates']
        datalist.append(indata['id']) # id data
        coord = indata['geometry']['coordinates']
        conv_data = coord # ConvUTM33NToWGS84(coord)
        datalist.append(conv_data)       # geometry
    else: # For weather
        keyslist = []
    
    tempdata = indata['properties']  # properties
    
    templist = list(tempdata.keys())
    for k in range(len(templist)):
        keyslist.append(templist[k])
        datalist.append(tempdata[templist[k]])
    #keyslist.append(templist[-1])
    #datalist.append(tempdata[templist[-1]])
 
    return keyslist, datalist

In [62]:
def loadfieldtoDF(tempdata, flag):
  #tempdata = allvalues[0]
  #print(tempdata)
  finalDF = pd.DataFrame()
  for k in tqdm(range(len(tempdata))): # each tempdata is a single field containing a number of time soil data
    #print(tempdata[k])
    #print("Progress : ",k,'/',len(tempdata))
    keylist, datalist =  ListfromData(tempdata[k], flag)
    tempDF  = ListstoDF(keylist, datalist)
    finalDF = finalDF.append(tempDF)
  return finalDF

In [63]:
# Converts dataframe to numpy array and saves it into binary file with name filename
def svpdtonpy(arrayin, filename):
  v = arrayin.reset_index()
  w=np.rec.fromrecords(v, names=v.columns.tolist())
  np.save(filename, w)

In [64]:
# Function to load all data from a specific field to a df
# input fielddata of a response from a json request 
# input pickafield containing a specific field number 
def LoadFieldDataToDF(fielddata, pickafield):
    
    singlefield = (fielddata["SRData"][0])["field"][pickafield]
    fieldcoord  = singlefield["geometry"]["coordinates"]#[0] # coord for last field number 
    convcoord   = convwgs84ToSWEREF99(fieldcoord[0])# Convert field coordinates to latt, long
    fieldprop = pd.DataFrame()
    
    fieldprop = loadfieldpropertiestoDF(singlefield) # DF containing all properties of that particular field
    fieldprop["geometry"] = [convcoord]
    fieldprop["id"] = singlefield["id"]
    #fieldprop.append(convcoord)
    #fieldprop.append(singlefield["id"])
    
    #fieldkeys.append("geometry")
    #fieldkeys.append("id")
                  
    return fieldprop #, fieldkeys # list containing all data from that particular field

In [42]:
# load numpy array filename and convert to Pandas dataframe
def loadnpy(filename):
    temparray = np.load(filename, allow_pickle=True)
    tempdf = pd.DataFrame(temparray)#, columns=temparray[0,1:])
    return tempdf

In [43]:
# check if df has same values in whole column
# Also remove any empty columns
def clean_df(dfin):
    for k in list(dfin.columns):
        if dfin[k].isnull().all()==True:
            dfin = dfin.drop(columns = [k])
            print("Dropping empty column ", str(k))
    return dfin 

# remove a column with a single value
def remov(dfin, col):
    firstvalue = dfin.iloc[0][col]
    if dfin[col].all() == firstvalue:
        print("All elements in column ", str(col), "have the same value:", firstvalue)
        print("dropped column ", str(col))
        dout = dfin.drop(columns=[col])
    else:
        print("There exist different values in column ", str(col), ". Keeping column intact.")
        dout = dfin
    return dout

# eliminates all data in provided column (not equal to given value)
# Creates new dataframe with data from column based on given value
def elim(dfin, col, value):
    dout = dfin[dfin[col]==value]
    print("kept only data which contain ", str(value), " in column ", str(col))
    return dout
#list_of_dfs['w'][list_of_dfs['w']['year'] == 2017]# harv, soil, w

In [44]:
# Clean weather dataframe 
def cleanDataFrame(dfin, cols_to_remove):
    tempdf = clean_df(dfin)
    dfout  = tempdf.drop(columns = cols_to_remove)#remove_columns)
    return dfout

# Convert date and time to just datetime object with only date
#format_string = "%Y-%m-%d"
#tempdf["time"] = pd.to_datetime(tempdf["time"], format = format_string)

## Setting up and initiating VPN 

In [46]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
        
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [47]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    
    return df

In [48]:
def calltoVPN(param_dic, textcom):
    
    # opening the VPN connection
    dbConnection = connect(param_dic) # Connect to PostgreSQL server
    
    start_time = datetime.datetime.now()        
    dfout = pd.read_sql_query(textcom, dbConnection)
    dif = datetime.datetime.now()-start_time
    print("Time to complete request:", dif.seconds, "sec.")
    
    # Close the database connection
    dbConnection.close();
    print("Connection closed")
    
    return dfout # output containing the response in a dataframe table

### Call to obtain a list of farms (blockids) in Heddinge with given properties

In [49]:
# Defining the different possible calls to be used later
def chose_a_call(argument,  region_name='Heddinge', crop_name='Höstvete',fieldID = 123456789):
    switcher = {
        # Gets blockids from cultivation - Heddinge only
        1: '''SELECT blockid FROM agri.cultivation WHERE farmname = 'Heddinge' ''',
        
        # Gets all from soil - specific field only
        2: '''SELECT * FROM agri.soil_sample u,agri.cult_union c \
        WHERE c.blockid = '''+str(fieldID),
        
        # Gets all from cultivation - specific: Varvete, Heddinge and seed_date
        3: '''SELECT * FROM agri.cultivation WHERE crop = 'Vårvete' \
        AND farmname = 'Heddinge' AND extract(month from seed_date) < 7''',
        
        # Gets all from cultivation - specific: Varvete, Heddinge and seed_date
        4: '''SELECT * FROM agri.cultivation WHERE crop = 'Vårkorn' \
        AND farmname = 'Heddinge' AND extract(month from seed_date) < 7\
        AND yield_harvester_measured > 0 AND humus_average < 10 ORDER BY blockid ''',
        
        # Gets all from cultivation - specific: Varvete, Heddinge and seed_date
        5: '''SELECT * FROM agri.cultivation WHERE crop = 'Höstvete' \
        AND farmname = 'Heddinge' AND extract(month from seed_date) < 7''',
        
        # unfinished call - do not use
        6: '''agri.cultivation SET crop = 'Höstvete' \
        WHERE crop ILIKE '%vete%' AND extract(month from seed_date) > 6;''',
        
        # Gets all fields from cultivation - specific: yield, humus
        7: '''SELECT * FROM agri.cultivation WHERE crop = 'Vårvete' \
        AND farmname = 'Heddinge' AND yield_harvester_measured > 0 AND humus_average < 10 ORDER BY blockid''',
        
        # This is latest command from Andreas about hostvete  #Vårkorn
        8: '''SELECT blockid FROM agri.cultivation WHERE crop = 'Höstvete' \
        AND farmname = ''' + region_name +''' AND yield_harvester_measured > 2500 \
        AND blockid IS NOT NULL AND seed_date IS NOT NULL;''',
        
        # This is latest command from Andreas about varkorn
        9: '''SELECT blockid, harvest_year FROM agri.cultivation WHERE crop = 'Vårkorn' \
        AND farmname = 'Heddinge' AND yield_harvester_measured > 2500 \
        AND blockid IS NOT NULL AND seed_date IS NOT NULL;''',
    }
    return switcher.get(argument,"Invalid choise")

## Calls to database 

In [207]:
# Defining the different possible calls to be used later
def new_calls(argument,  region_name = "'Heddinge'", crop_name="'Vårvete'",fieldID = 123456789):
    switcher = {
        
        # This is for weather 
        0: #"Select * FROM agri.weather ORDER BY year ASC",#w.temperature AS w_temperature, \
        "SELECT w.precipitation AS w_precipitation, \
        w.year AS w_year, \
        w.time AS w_time \
        FROM agri.cultivation c,agri.weather w \
        WHERE c.blockid = " + str(fieldID)+ "\
        AND extract(month from w.time) < 11 \
        AND extract(month from w.time) > 2 \
        ORDER BY year ASC",
        
        # This is for harvest
        1:
        "SELECT ST_X(u.geom) AS h_coord_X, ST_Y(u.geom) AS h_coord_Y, \
        c.blockid AS h_blockid, \
        u.year AS h_year, \
        u.time AS h_time, \
        u.harvest AS h_harvest, \
        u.water_percentage AS h_water_percentage \
        FROM agri.harvest u, agri.cult_union c WHERE c.blockid = "+str(fieldID)+" \
        AND ST_Intersects(c.geom, u.geom) \
        AND u.crop = 'Vete' \
        AND u.edgeflag = 'False' \
        ORDER BY year,id ASC",
        #+ crop_name +" \
        #
        
        # This is for soil
        2: #'''SELECT ST_X(u.geom) AS coord_X, ST_Y(u.geom) AS coord_Y FROM agri.soil_sample u,agri.cult_union c WHERE c.blockid = '''+str(fieldID),
        '''SELECT ST_X(u.geom) AS coord_X, ST_Y(u.geom) AS coord_Y, \
        u.ph AS s_ph, \
        u.humus_percentage AS s_hum100, \
        u.clay_percentage AS s_clay100, \
        u.phosphorus AS s_phosphorus, \
        u.calcium AS s_calcium, \
        u.potassium AS s_K, u.magnesium AS s_mg \
        FROM agri.soil_sample u,agri.cult_union c WHERE c.blockid = '''+str(fieldID)+''' \
        AND ST_IsValid(c.geom) \
        AND ST_Intersects(c.geom,u.geom) \
        AND u.youngest = 'True' ORDER BY year,id ASC''',
        #
        
        
        # This is for field
        3:'''SELECT \
        blockid AS f_blockid,\
        seed_date AS f_seed_date, \
        seed_amount AS f_seed_amount, \
        harvest_date AS f_harvest_date, \
        yield_harvester_measured AS f_yield_harvester_measured, \
        yield_water_percentage AS f_yield_water_percentage, \
        fertilizer_date AS f_fertilizer_date, \
        fertilizer_amount AS f_fertilizer_amount, \
        humus_average AS f_humus_average, \
        humus_stddev AS f_humus_stddev,\
        ph_average AS f_ph_average, \
        ph_stddev AS f_ph_stddev, \
        weatherstation AS f_weatherstation, \
        weatherstation_dist AS f_weatherstation_dist, \
        blockid_org AS f_blockid_org, \
        nitrogen AS f_nitrogen, \
        kalium AS f_kalium, \
        phosphorus AS f_phosphorus, \
        sulphur AS f_sulphur, \
        calcium AS f_calcium, \
        magnesium AS f_MG, \
        nitrogen_earlier AS f_nitrogen_earlier, \
        edgeflag_percentage AS f_edgeflag_percentage \
        FROM agri.cultivation  \
        WHERE crop = ''' + crop_name +''' \
        AND farmname = 'Heddinge' \
        AND blockID = '''+str(fieldID),
        # slopeness AS f_slopeness, \
        #nitrogen_secondary AS f_nitrogen_secondary, \
        #weather_precipitation AS f_weather_precip, \
        #weather_temperature AS f_weather_temp, \
        #harvest_year AS f_har_year, \
        #crop AS f_crop, \
        #geom AS f_geom, \
        #yield_amount AS f_yield_amount, \
        #''' + region_name +''' \
        # Gets all from soil_sample - specific: youngest, fieldID
        
        5:'''SELECT * FROM agri.soil_sample WHERE youngest = 'True' \
        AND c.blockid = '''+str(fieldID),
        
        # Unfinished call - do not use
        10: "SELECT a.id,a.year,b.id,b.year,ST_X(a.geom),ST_X(b.geom) \
        FROM agri.soil_sample a,agri.soil_sample b WHERE ST_DWithin(a.geom,b.geom,10) \
        AND a.id <> b.id AND a.year <> b.year ORDER BY ST_X(a.geom),a.year;"
    }
    return switcher.get(argument,"Invalid choise - by myself")

# WHERE crop ILIKE '%vete%'  

## Grouping weather data by Seasons, Month, week - do a total 

Total average of all columns

In [ ]:
# REMOVE?!

# Defining the different possible calls to be used later
def choose_group(argument):
    switcher = {
        # Gets blockids from cultivation - Heddinge only
        "D": '''Nothin.... ''',
        
        # Gets all from soil - specific field only
        "W": "nothing",
        
        # Gets all from cultivation - specific: Varvete, Heddinge and seed_date
        "M": '''nothing ...'''

    }
    return switcher.get(argument,"Invalid choise. \
    Please choose from 'D', 'W' or 'M' for day, week or month respectively ")

In [154]:
def cutoutseason(datafrin, seasonb, seasone):
    
    datafrout = datafrin[datafrin["w_time"]  >seasonb]
    datafrout = datafrout[datafrout["w_time"]<seasone]
    
    return datafrout

In [155]:
# Defining the different possible groupings to choose from 
# 0: group all weather data into a single group
# 1: group all weather data per week
# 2: group all weather data per month
# 3: group all weather data per specific dates

def group_weather(argument, tempdf, fieldf, year_chosen):#, fieldID = 123456789):
    
    if argument == 0:   # average all data into a single group (as an average)
        ntempdf = pd.DataFrame(columns=tempdf.columns)
        ntempdf.loc['mean'] = tempdf.mean() # do an average over the total
        
    elif argument == 1: # group weather data per week (as an average)
        
        format_string = "%Y-%m-%d"
        tempdf["w_time"] = pd.to_datetime(tempdf["w_time"], format = format_string)

        # choose to keep year 2020
        tempdf = elim(tempdf, "w_year", year_chosen)
        
        groupby = "W"  # group by week, (M) Month. Alternatively "D" for day
        ntempdf = tempdf.groupby(pd.Grouper(key='w_time',freq = groupby)).mean()
        
    elif argument == 2: # group weather data per month (as an average)
        # First change the date column to datetime object
        # Convert date and time to just datetime object with only date
        format_string = "%Y-%m-%d"
        tempdf["w_time"] = pd.to_datetime(tempdf["w_time"], format = format_string)

        # choose to keep year 2020
        tempdf = elim(tempdf, "w_year", year_chosen)

        groupby = "M"  # group by week, (M) Month. Alternatively "D" for day
        ntempdf = tempdf.groupby(pd.Grouper(key='w_time',freq = groupby)).mean()
        
    elif argument == 3: # average over specific seasons 
        # First create a format for indexing seed date
        #format_string = "%Y-%m-%d"
        #nfieldf["w_time"] = pd.to_datetime(nfieldf["f_seed_date"], format = format_string)
        #fieldf["f_harvest_date"] = pd.to_datetime(fieldf["f_harvest_date"], format = format_string)
        #print("Full field dataset was ", nfieldf.shape)
        #print("Number of fields seeded in ", year_chosen-1," were: ", \
        #      len(nfieldf[nfieldf["f_seed_date"]<datetime.date(year_chosen-1,10,30)]) )

        print("The following will be used as a start time for weather grouping.")
        # Find minimum date of seeding
        min_seed_date = min(fieldf[fieldf["f_seed_date"]>datetime.date(year_chosen-1,9,2)]["f_seed_date"])#.shape
        print("The earliest seeding date is ", min_seed_date)

        # Get harvest date for this dataset
        max_harv_date = max(fieldf[fieldf["f_harvest_date"]<datetime.date(year_chosen,9,1)]["f_harvest_date"])
        print("MAximum harvest time: ", max_harv_date)
        #max_harv_date = max(fieldf[fieldf["f_harvest_date"]>datetime.date(year_chosen,7,1)]["f_seed_date"])

        # Season 1: Seed-start-> 1st Nov
        season1st  = min_seed_date
        season1end = datetime.date(year_chosen-1,11,1) # November 1st
        # Season 2: 15 March  -> Midsummer
        season2st  = datetime.date(year_chosen,3,15)   # March 15
        season2end = datetime.date(year_chosen,6,25)   # June 25 (midsummer)
        # Season 3: Midsummer -> Harvest date
        season3st  = datetime.date(year_chosen,6,26)   # June 26 (after midsummer)
        season3end = max_harv_date
        
        print("Dates for season 1: ", season1st, season1end)
        print("Dates for season 2: ", season2st, season2end)
        print("Dates for season 3: ", season3st, season3end)

        t1 = cutoutseason(tempdf, season1st, season1end)
        if len(t1) == 0:
            print("Problems!!!")
            t1
            
        t1["mean"]=t1.mean()
        t2 = cutoutseason(tempdf, season2st, season2end)
        t3 = cutoutseason(tempdf, season3st, season3end)
        #ntempdf = pd.DataFrame(columns=tempdf.columns)
        #t1 = tempdf[tempdf["f_harvest_date"]]>season1st
        #t2 = t1[t1["f_harvest_date"]]<season1end
        ntempdf = pd.DataFrame()#columns=["w_precipitation"])
        l = pd.Series([t1.mean()["w_precipitation"], t2.mean()["w_precipitation"], t3.mean()["w_precipitation"]])
        ntempdf["w_precipitation"] = l
        #ntempdf # this is weather grouped by season
        
    
    return ntempdf

### Now work on spatial proximity grouping between datasets

First with Harvest: for a given square with side length 2r we group together (sum) all harvest data which are within that distance for each coordinate in the soil data.

In [83]:
# Creates lists of x and y coordinates from dataframes. 
def createdftoxylist1(nnharvdf, nnsoildf):
    
    xsoil_coord = list(nnsoildf["coord_x"])   # x Soil coordinates 
    ysoil_coord = list(nnsoildf["coord_y"])   # y Soil coordinates 

    xharv_coord = list(nnharvdf["h_coord_x"]) # x Harvest coordinates
    yharv_coord = list(nnharvdf["h_coord_y"]) # y Harvest coordinates
    
    return xharv_coord, yharv_coord, xsoil_coord, ysoil_coord

def remdata(nnharvdf, nnsoildf, r=10):
    new_soil_df = nnsoildf.sort_values("coord_x") # sort soil dataframe based on x_coordinate
    nnharvdf = nnharvdf.sort_values("h_coord_x") # sort soil dataframe based on x_coordinate
    xharv_coord, yharv_coord, xsoil_coord, ysoil_coord = createdftoxylist(nnharvdf, new_soil_df)
    new_harv_df = pd.DataFrame(columns=nnharvdf.keys())             # empty dataframe
    
    remind = [] # index of rows from harvest to be removed at end
    
    # HARVEST first
    for s in tqdm(range(len(xharv_coord))): # Looping through all harvest coordinates
        x_center = xharv_coord[s]
        y_center = yharv_coord[s]

        xlow = x_center-r 
        xhigh= x_center+r
        ylow = y_center-r
        yhigh= y_center+r    
        cnt3=0
        cnt2=0
        
        # SOIL next
        for indx in range(len(xsoil_coord)):# Looping through all soil coordinates
            sumind = [] # index of rows from soil to add-up later at end
            xh = xharv_coord[indx]  # x soil coordinate
            
            if xh>xlow: # if within r of the x harv coordinate  then look at the y coordinate as well
                yh = yharv_coord[indx]  # y soil coordinate
                if yh>ylow and yh<yhigh: # we keep this index!
                    sumind.append(indx) # indexes to sum up in soil
                    cnt3 = cnt3+1
                    #print("Soil", xh, "inside circle centered at harvest:", x_center)
            if xh>xhigh:
                break # we are finished with this harv coordinate- move on to next soil coordinate
                
        if len(sumind) == 0: # i.e. no points within range then remove that index from soil
            remind.append(indx) # indexes to remove from harvest
        else:
            new_soil_df.loc[cnt2]=list(nnsoildf.iloc[sumind,:].sum()/cnt3)  # average soil indexes
            cnt2 = cnt2+1
            
    new_harv_df=nnharvdf.drop(remind)             # remove harvest indexes
    
    return new_harv_df, new_soil_df

# For each point in the soil find all points in harvest within given radius
# Once points found then we average all the soil contributions (within circle) into input vector dataframe grouped around those points
# It creates a new harvest dataframe with same number of rows as those from soil dataframe
def groupSoil(nnharvdf, nnsoildf, r=10):
    #r = 10 # Radius is ad-hoc and given by user
    
    xharv_coord, yharv_coord, xsoil_coord, ysoil_coord = createdftoxylist(nnharvdf, nnsoildf)
    
    cnt2 = 0
    for s in tqdm(range(len(xsoil_coord))): # Looping through all soil coordinates
        print("at soil coordinate: ",s)
        x_center = xsoil_coord[s]#geometry.Point(s)#[0],s[1])  # Note: center is given by soil coordinates!
        y_center = ysoil_coord[s]

        cnt3 = 0
        rowind = [] # index of rows in harvest dataframe to be added together later on
        xlow = x_center-r 
        xhigh= x_center+r
        ylow = y_center-r
        yhigh= y_center+r

        for indx in range(len(xharv_coord)):#harv_coord: # Looping through all harvest coordinates
            xh = xharv_coord[indx]

            if xh>xlow: # if within r of the x soil coordinate  then look at the y coordinate as well
                yh = yharv_coord[indx]
                if yh>ylow and yh<yhigh: # we keep this index!
                    rowind.append(indx)
                    cnt3 = cnt3+1
                    #print("Harvest", xh, "inside circle centered at soil:", x_center)
            if xh>xhigh:
                break # we are finished with this soil coordinate- move on to next soil coordinate

        #print("Added ", cnt3, " harvest data within this soil center." )
        if rowind == []:
            print("             PROBLE<S!!     No harvest data near this soil coordinate ")
        newdf.loc[cnt2]=list(newharvdf.iloc[rowind,:].sum()/cnt3)  # average harvest
        #print("item included is index:",cnt2," with value ", nnharvdf.iloc[rowind,:].sum()/cnt3)
        cnt2 = cnt2+1
        
    return newdf # Harvest DataFrame grouped according to Soil coordinates (same number of row - in same order)

In [ ]:
# Creates lists of x and y coordinates from dataframes. 
def h_a_h_createdftoxylist(nnharvdf, nnsoildf):
    
    xsoil_coord = list(nnsoildf["h_coord_x"])   # x Soil coordinates 
    ysoil_coord = list(nnsoildf["h_coord_y"])   # y Soil coordinates 

    xharv_coord = list(nnharvdf["h_coord_x"]) # x Harvest coordinates
    yharv_coord = list(nnharvdf["h_coord_y"]) # y Harvest coordinates
    
    return xharv_coord, yharv_coord, xsoil_coord, ysoil_coord

In [ ]:
def harvest_around_harvest(nnharvdf, nnsoildf, r=10):
    new_soil_df = nnsoildf.sort_values("h_coord_x") # sort soil dataframe based on x_coordinate
    nnharvdf = nnharvdf.sort_values("h_coord_x") # sort soil dataframe based on x_coordinate
    xharv_coord, yharv_coord, xsoil_coord, ysoil_coord = h_a_h_createdftoxylist(nnharvdf, new_soil_df)
    new_harv_df = pd.DataFrame(columns=nnharvdf.keys())             # empty dataframe
    
    remind = [] # index of rows from harvest to be removed at end
    cnt2=0
    
    # SOIL first
    for xs in tqdm(range(len(xsoil_coord))): # Looping through all soil coordinates
        x_center = xsoil_coord[xs]
        y_center = ysoil_coord[xs]

        xlow = x_center-r 
        xhigh= x_center+r
        ylow = y_center-r
        yhigh= y_center+r    
        cnt3=0

        sumind = [] # index of rows from soil to add-up later at end
        
        # HARVEST next
        for xind in range(len(xharv_coord)):# Looping through all soil coordinates
            
            xh = xharv_coord[xind]  # x soil coordinate
            
            if xh>xlow: # if within r of the x harv coordinate  then look at the y coordinate as well
                yh = yharv_coord[xind]  # y soil coordinate
                if yh>ylow and yh<yhigh: # we keep this index!
                    sumind.append(xind) # indexes to sum up in soil
                    cnt3 = cnt3+1
                    #print("Soil", xh, "inside circle centered at harvest:", x_center)
            if xh>xhigh:
                break # we are finished with this harv coordinate- move on to next soil coordinate
                
        if len(sumind) == 0: # i.e. no points within range then remove that index from soil
            remind.append(xs) # indexes to remove from soil
            cn2 = cnt2+1
        else:
            new_harv_df.loc[cnt2]=list(nnharvdf.iloc[sumind,:].sum()/cnt3)  # average harv indexes
            cnt2 = cnt2+1
    df = new_soil_df
    df.reset_index(drop=True, inplace=True)
    df.drop(df.index[remind], inplace=True)
    new_soil_df=df #new_soil_df.drop(remind)             # remove harvest indexes
    #print("Soil shape", new_soil_df.shape)
    print("New Harvest shape", new_harv_df.shape)
    return new_harv_df #, new_soil_df

In [ ]:
# Creates lists of x and y coordinates from dataframes. 
def s_a_h_createdftoxylist(nnharvdf, nnsoildf):
    
    xsoil_coord = list(nnsoildf["h_coord_x"])   # x Soil coordinates 
    ysoil_coord = list(nnsoildf["h_coord_y"])   # y Soil coordinates 

    xharv_coord = list(nnharvdf["coord_x"]) # x Harvest coordinates
    yharv_coord = list(nnharvdf["coord_y"]) # y Harvest coordinates
    
    return xharv_coord, yharv_coord, xsoil_coord, ysoil_coord

In [ ]:
def soil_around_harvest(nnharvdf, nnsoildf, r=10):
    new_soil_df = nnsoildf.sort_values("h_coord_x") # sort soil dataframe based on x_coordinate
    nnharvdf = nnharvdf.sort_values("coord_x") # sort soil dataframe based on x_coordinate
    xharv_coord, yharv_coord, xsoil_coord, ysoil_coord = s_a_h_createdftoxylist(nnharvdf, new_soil_df)
    new_harv_df = pd.DataFrame(columns=nnharvdf.keys())             # empty dataframe
    
    remind = [] # index of rows from harvest to be removed at end
    cnt2=0
    
    # SOIL first
    for xs in tqdm(range(len(xsoil_coord))): # Looping through all soil coordinates
        x_center = xsoil_coord[xs]
        y_center = ysoil_coord[xs]

        xlow = x_center-r 
        xhigh= x_center+r
        ylow = y_center-r
        yhigh= y_center+r    
        cnt3=0

        sumind = [] # index of rows from soil to add-up later at end
        
        # HARVEST next
        for xind in range(len(xharv_coord)):# Looping through all soil coordinates
            
            xh = xharv_coord[xind]  # x soil coordinate
            
            if xh>xlow: # if within r of the x harv coordinate  then look at the y coordinate as well
                yh = yharv_coord[xind]  # y soil coordinate
                if yh>ylow and yh<yhigh: # we keep this index!
                    sumind.append(xind) # indexes to sum up in soil
                    cnt3 = cnt3+1
                    #print("Soil", xh, "inside circle centered at harvest:", x_center)
            if xh>xhigh:
                break # we are finished with this harv coordinate- move on to next soil coordinate
                
        if len(sumind) == 0: # i.e. no points within range then remove that index from soil
            remind.append(xs) # indexes to remove from soil
            cn2 = cnt2+1
        else:
            new_harv_df.loc[cnt2]=list(nnharvdf.iloc[sumind,:].sum()/cnt3)  # average harv indexes
            cnt2 = cnt2+1
    df = new_soil_df
    df.reset_index(drop=True, inplace=True)
    df.drop(df.index[remind], inplace=True)
    new_soil_df=df #new_soil_df.drop(remind)             # remove harvest indexes
    print("Soil shape", new_soil_df.shape)
    print("Harv shape", new_harv_df.shape)
    return new_harv_df, new_soil_df

In [ ]:
# Creates lists of x and y coordinates from dataframes. 
def h_a_s_createdftoxylist(nnharvdf, nnsoildf):
    
    xsoil_coord = list(nnsoildf["coord_x"])   # x Soil coordinates 
    ysoil_coord = list(nnsoildf["coord_y"])   # y Soil coordinates 

    xharv_coord = list(nnharvdf["h_coord_x"]) # x Harvest coordinates
    yharv_coord = list(nnharvdf["h_coord_y"]) # y Harvest coordinates
    
    return xharv_coord, yharv_coord, xsoil_coord, ysoil_coord

In [84]:
def harvest_around_soil(nnharvdf, nnsoildf, r=10):
    new_soil_df = nnsoildf.sort_values("coord_x") # sort soil dataframe based on x_coordinate
    nnharvdf = nnharvdf.sort_values("h_coord_x") # sort soil dataframe based on x_coordinate
    xharv_coord, yharv_coord, xsoil_coord, ysoil_coord = h_a_s_createdftoxylist(nnharvdf, new_soil_df)
    new_harv_df = pd.DataFrame(columns=nnharvdf.keys())             # empty dataframe
    
    remind = [] # index of rows from harvest to be removed at end
    cnt2=0
    
    # SOIL first
    for xs in tqdm(range(len(xsoil_coord))): # Looping through all soil coordinates
        x_center = xsoil_coord[xs]
        y_center = ysoil_coord[xs]

        xlow = x_center-r 
        xhigh= x_center+r
        ylow = y_center-r
        yhigh= y_center+r    
        cnt3=0

        sumind = [] # index of rows from soil to add-up later at end
        
        # HARVEST next
        for xind in range(len(xharv_coord)):# Looping through all soil coordinates
            
            xh = xharv_coord[xind]  # x soil coordinate
            
            if xh>xlow: # if within r of the x harv coordinate  then look at the y coordinate as well
                yh = yharv_coord[xind]  # y soil coordinate
                if yh>ylow and yh<yhigh: # we keep this index!
                    sumind.append(xind) # indexes to sum up in soil
                    cnt3 = cnt3+1
                    #print("Soil", xh, "inside circle centered at harvest:", x_center)
            if xh>xhigh:
                break # we are finished with this harv coordinate- move on to next soil coordinate
                
        if len(sumind) == 0: # i.e. no points within range then remove that index from soil
            remind.append(xs) # indexes to remove from soil
            cn2 = cnt2+1
        else:
            new_harv_df.loc[cnt2]=list(nnharvdf.iloc[sumind,:].sum()/cnt3)  # average harv indexes
            cnt2 = cnt2+1
    df = new_soil_df
    df.reset_index(drop=True, inplace=True)
    df.drop(df.index[remind], inplace=True)
    new_soil_df=df #new_soil_df.drop(remind)             # remove harvest indexes
    print("Soil shape", new_soil_df.shape)
    print("Harv shape", new_harv_df.shape)
    return new_harv_df, new_soil_df

In [1]:
# Evaluation Functions
def rmse(model, y_test, y_pred, X_train, y_train):
 r_squared = model.score(X_test, y_test)
 mse = mean_squared_error(y_test, y_pred)
 rmse = np.sqrt(mse)
 print('R-squared: ' + str(r_squared))
 print('Mean Squared Error: '+ str(rmse))

# Create model line scatter plot
def scatter_plot(y_test, y_pred, model_name):
    plt.figure(figsize=(10,6))
    sns.residplot(y_test, y_pred, lowess=True, color='#4682b4',
              line_kws={'lw': 2, 'color': 'r'})
    plt.title(str('Price vs Residuals for '+ model_name))
    plt.xlabel('Price',fontsize=16)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.show()

In [ ]:
def heatmap(x, y, **kwargs):
    if 'color' in kwargs:
        color = kwargs['color']
    else:
        color = [1]*len(x)

    if 'palette' in kwargs:
        palette = kwargs['palette']
        n_colors = len(palette)
    else:
        n_colors = 256 # Use 256 colors for the diverging color palette
        palette = sns.color_palette("Blues", n_colors) 

    if 'color_range' in kwargs:
        color_min, color_max = kwargs['color_range']
    else:
        color_min, color_max = min(color), max(color) # Range of values that will be mapped to the palette, i.e. min and max possible correlation

    def value_to_color(val):
        if color_min == color_max:
            return palette[-1]
        else:
            val_position = float((val - color_min)) / (color_max - color_min) # position of value in the input range, relative to the length of the input range
            val_position = min(max(val_position, 0), 1) # bound the position betwen 0 and 1
            ind = int(val_position * (n_colors - 1)) # target index in the color palette
            return palette[ind]

    if 'size' in kwargs:
        size = kwargs['size']
    else:
        size = [1]*len(x)

    if 'size_range' in kwargs:
        size_min, size_max = kwargs['size_range'][0], kwargs['size_range'][1]
    else:
        size_min, size_max = min(size), max(size)

    size_scale = kwargs.get('size_scale', 500)

    def value_to_size(val):
        if size_min == size_max:
            return 1 * size_scale
        else:
            val_position = (val - size_min) * 0.99 / (size_max - size_min) + 0.01 # position of value in the input range, relative to the length of the input range
            val_position = min(max(val_position, 0), 1) # bound the position betwen 0 and 1
            return val_position * size_scale
    if 'x_order' in kwargs: 
        x_names = [t for t in kwargs['x_order']]
    else:
        x_names = [t for t in sorted(set([v for v in x]))]
    x_to_num = {p[1]:p[0] for p in enumerate(x_names)}

    if 'y_order' in kwargs: 
        y_names = [t for t in kwargs['y_order']]
    else:
        y_names = [t for t in sorted(set([v for v in y]))]
    y_to_num = {p[1]:p[0] for p in enumerate(y_names)}

    plot_grid = plt.GridSpec(1, 15, hspace=0.2, wspace=0.1) # Setup a 1x10 grid
    ax = plt.subplot(plot_grid[:,:-1]) # Use the left 14/15ths of the grid for the main plot

    marker = kwargs.get('marker', 's')

    kwargs_pass_on = {k:v for k,v in kwargs.items() if k not in [
         'color', 'palette', 'color_range', 'size', 'size_range', 'size_scale', 'marker', 'x_order', 'y_order'
    ]}

    ax.scatter(
        x=[x_to_num[v] for v in x],
        y=[y_to_num[v] for v in y],
        marker=marker,
        s=[value_to_size(v) for v in size], 
        c=[value_to_color(v) for v in color],
        **kwargs_pass_on
    )
    ax.set_xticks([v for k,v in x_to_num.items()])
    ax.set_xticklabels([k for k in x_to_num], rotation=45, horizontalalignment='right')
    ax.set_yticks([v for k,v in y_to_num.items()])
    ax.set_yticklabels([k for k in y_to_num])

    ax.grid(False, 'major')
    ax.grid(True, 'minor')
    ax.set_xticks([t + 0.5 for t in ax.get_xticks()], minor=True)
    ax.set_yticks([t + 0.5 for t in ax.get_yticks()], minor=True)

    ax.set_xlim([-0.5, max([v for v in x_to_num.values()]) + 0.5])
    ax.set_ylim([-0.5, max([v for v in y_to_num.values()]) + 0.5])
    ax.set_facecolor('#F1F1F1')

    # Add color legend on the right side of the plot
    if color_min < color_max:
        ax = plt.subplot(plot_grid[:,-1]) # Use the rightmost column of the plot

        col_x = [0]*len(palette) # Fixed x coordinate for the bars
        bar_y=np.linspace(color_min, color_max, n_colors) # y coordinates for each of the n_colors bars

        bar_height = bar_y[1] - bar_y[0]
        ax.barh(
            y=bar_y,
            width=[5]*len(palette), # Make bars 5 units wide
            left=col_x, # Make bars start at 0
            height=bar_height,
            color=palette,
            linewidth=0
        )
        ax.set_xlim(1, 2) # Bars are going from 0 to 5, so lets crop the plot somewhere in the middle
        ax.grid(False) # Hide grid
        ax.set_facecolor('white') # Make background white
        ax.set_xticks([]) # Remove horizontal ticks
        ax.set_yticks(np.linspace(min(bar_y), max(bar_y), 3)) # Show vertical ticks for min, middle and max
        ax.yaxis.tick_right() # Show vertical ticks on the right 


def corrplot(data, size_scale=500, marker='s'):
    corr = pd.melt(data.reset_index(), id_vars='index')
    corr.columns = ['x', 'y', 'value']
    heatmap(
        corr['x'], corr['y'],
        color=corr['value'], color_range=[-1, 1],
        palette=sns.diverging_palette(20, 220, n=256),
        size=corr['value'].abs(), size_range=[0,1],
        marker=marker,
        x_order=data.columns,
        y_order=data.columns[::-1],
        size_scale=size_scale
    )
